# Tutorial 2.2: Inputs, Files, and Scans

## 2.2.0: Python Imports

In [1]:
%load_ext autoreload
%autoreload 2

Please import:

In [121]:
import numpy

from mbfit.utils.filesystem import Directory
from mbfit.utils.job_runner import LocalJobRunner, SLURMJobRunner

from mbfit.system.system_like import BondType
from mbfit.system.definition import BasicFragmentDefinition, BasicCompoundDefinition, AtomDefinition, BondDefinition
from mbfit.system.initializer import VSEPRInitializer
from mbfit.system.descriptor import fragment_definition_to_xyz, compound_definition_to_xyz, compound_to_xyz
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system import BasicCompound
from mbfit.system.transformer import SystemTransformer
from mbfit.system.copier import CompoundCopier
from mbfit.system.transformer import make_scan
from mbfit.system.region import BasicMutableRegionManager

## 2.2.1 Directory Structure & File Paths

In this section, we we will setup the directory structure and file paths we will use in this notebook. Throughout this notebook, the following files will be created:

In [3]:
tutorial_2_workdir = Directory("Tutorial_2_workdir")
tutorial_2_workdir.create()

In [4]:
definitions_dir = tutorial_2_workdir.sub_directory("definitions")
definitions_dir.create()

In [5]:
fragment_CH3NH2_definition_path = definitions_dir.file("fragment_CH3NH2.def")
fragment_H2O_definition_path = definitions_dir.file("fragment_H2O.def")

In [6]:
monomer_CH3NH2_definition_path = definitions_dir.file("monomer_CH3NH2.def")
monomer_H2O_definition_path = definitions_dir.file("monomer_H2O.def")
dimer_CH3NH2_H2O_definition_path = definitions_dir.file("dimer_CH3NH2_H2O.def")
trimer_CH3NH2_H2O_H2O_definition_path = definitions_dir.file("trimer_CH3NH2_H2O_H2O.def")

In [7]:
structures_dir = tutorial_2_workdir.sub_directory("structures")
structures_dir.create()

In [8]:
monomer_CH3NH2_initial_structure_path = structures_dir.file("monomer_CH3NH2_initial_structure.xyz")
monomer_H2O_initial_structure_path = structures_dir.file("monomer_H2O_initial_structure.xyz")

monomer_CH3NH2_minimum_structure_path = structures_dir.file("monomer_CH3NH2_minimum_structure.xyz")
monomer_H2O_minimum_structure_path = structures_dir.file("monomer_H2O_minimum_structure.xyz")

In [74]:
scans_dir = tutorial_2_workdir.sub_directory("scans")
scans_dir.create()

In [119]:
scan_2b_1_path = scans_dir.file("scan_2b_1.xyz")
scan_2b_2_path = scans_dir.file("scan_2b_2.xyz")
scan_2b_3_path = scans_dir.file("scan_2b_3.xyz")

scan_3b_1_path = scans_dir.file("scan_3b_1.xyz")
scan_3b_2_path = scans_dir.file("scan_3b_2.xyz")
scan_3b_3_path = scans_dir.file("scan_3b_3.xyz")

In [143]:
scan_2b_energies_1_path = scans_dir.file("scan_2b_1.xyz")
scan_2b_energies_2_path = scans_dir.file("scan_2b_2.xyz")
scan_2b_energies_3_path = scans_dir.file("scan_2b_3.xyz")

scan_3b_energies_1_path = scans_dir.file("scan_3b_1.xyz")
scan_3b_energies_2_path = scans_dir.file("scan_3b_2.xyz")
scan_3b_energies_3_path = scans_dir.file("scan_3b_3.xyz")

In [9]:
logs_dir = tutorial_2_workdir.sub_directory("logs")
logs_dir.create()

In [141]:
job_runner = SLURMJobRunner(
        tutorial_2_workdir.sub_directory("jobs"),
        """#!/bin/bash

#SBATCH --job-name="{job_name}"
#SBATCH --output="compile.stdout"
#SBATCH --partition=shared
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --mem=32GB
#SBATCH -A csd888
#SBATCH --export=ALL
#SBATCH -t 01:00:00

module load cpu/0.15.4  gcc/10.2.0  mvapich2/2.3.6 qchem/5.4
conda activate p310
cd ~/software/development/MB-Fit_versions/QDMS_2025
export MBFIT_HOME=$PWD
source sourceme.sh
cd -

python -u {script_path} > {output_path} 2>&1
"""
)

#job_runner = LocalJobRunner(tutorial_2_workdir.sub_directory("jobs"))

In [142]:
orca_runner = SLURMJobRunner(
        tutorial_2_workdir.sub_directory("jobs"),
        """#!/bin/bash

#SBATCH --job-name="{job_name}"
#SBATCH --output="compile.stdout"
#SBATCH --partition=shared
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --mem=32GB
#SBATCH -A csd888
#SBATCH --export=ALL
#SBATCH -t 01:00:00

module load cpu/0.17.3b  gcc/10.2.0/npcyll4  openmpi/4.1.1/ygduf2r
module load orca/5.0.4
export ORCA_SCRATCH=/scratch/$USER/job_$SLURM_JOB_ID
conda activate p310
cd ~/software/development/MB-Fit_versions/QDMS_2025
export MBFIT_HOME=$PWD
source sourceme.sh
cd -

python -u {script_path} > {output_path} 2>&1
"""
)

#job_runner = LocalJobRunner(tutorial_2_workdir.sub_directory("jobs"))

## 2.2.x Create `.def` definition files

First, we will create the two fragment definition files. First, the definition for the CH3NH2 fragment:

In [11]:
fragment_CH3NH2_definition = BasicFragmentDefinition()
CA  = fragment_CH3NH2_definition.add_atom(AtomDefinition("C", "A"))
HB1 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "B"))
HB2 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "B"))
HB3 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "B"))
NC  = fragment_CH3NH2_definition.add_atom(AtomDefinition("N", "C"))
HD1 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "D"))
HD2 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "D"))
fragment_CH3NH2_definition.add_bond(CA, HB1, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(CA, HB2, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(CA, HB3, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(CA, NC, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(NC, HD1, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(NC, HD2, BondDefinition(BondType.SINGLE))

fragment_definition_to_xyz.writer(fragment_CH3NH2_definition_path, fragment_CH3NH2_definition)

The definition for the H2O fragment:

In [12]:
fragment_H2O_definition = BasicFragmentDefinition()
OE  = fragment_H2O_definition.add_atom(AtomDefinition("O", "E"))
HF1 = fragment_H2O_definition.add_atom(AtomDefinition("H", "F"))
HF2 = fragment_H2O_definition.add_atom(AtomDefinition("H", "F"))
fragment_H2O_definition.add_bond(OE, HF1, BondDefinition(BondType.SINGLE))
fragment_H2O_definition.add_bond(OE, HF2, BondDefinition(BondType.SINGLE))

fragment_definition_to_xyz.writer(fragment_H2O_definition_path, fragment_H2O_definition)

Next, we will make three n-mer definitions.

First, the CH3NH2 monomer, which has a single CH3NH2 fragment.

In [13]:
monomer_CH3NH2_definition = BasicCompoundDefinition()
monomer_CH3NH2_definition.add_fragment(fragment_CH3NH2_definition)

compound_definition_to_xyz.writer(monomer_CH3NH2_definition_path, monomer_CH3NH2_definition)

First, the H2O monomer, which has a single CH3NH2 fragment.

In [14]:
monomer_H2O_definition = BasicCompoundDefinition()
monomer_H2O_definition.add_fragment(fragment_H2O_definition)

compound_definition_to_xyz.writer(monomer_H2O_definition_path, monomer_H2O_definition)

Next, the CH3NH2 -- H2O dimer, with two fragments:

In [15]:
dimer_CH3NH2_H2O_definition = BasicCompoundDefinition()
dimer_CH3NH2_H2O_definition.add_fragment(fragment_CH3NH2_definition)
dimer_CH3NH2_H2O_definition.add_fragment(fragment_H2O_definition)

compound_definition_to_xyz.writer(dimer_CH3NH2_H2O_definition_path, dimer_CH3NH2_H2O_definition)

Finally, the CH3NH2 -- H2O -- H2O trimer, with three fragments:

In [16]:
trimer_CH3NH2_H2O_H2O_definition = BasicCompoundDefinition()
trimer_CH3NH2_H2O_H2O_definition.add_fragment(fragment_CH3NH2_definition)
trimer_CH3NH2_H2O_H2O_definition.add_fragment(fragment_H2O_definition)
trimer_CH3NH2_H2O_H2O_definition.add_fragment(fragment_H2O_definition)

compound_definition_to_xyz.writer(trimer_CH3NH2_H2O_H2O_definition_path, trimer_CH3NH2_H2O_H2O_definition)

We will use these definition files throughout Tutorial 2.

## 2.2.x Geometry Initialization

Next, we need a reasonable initial guess for the structure of our monomer. The guess need only be good enough that it will covnerge to the actual minimum structure when we perform a minimization with DFT. MB-Fit contains the class `VESPRInitializer`, which will make a reasonable guess about the structure of a molecule based on its bonding topology. Let's use `VESPRInitializer` to create an initial structure file:

In [17]:
definition = xyz_to_compound_definition.reader(monomer_CH3NH2_definition_path)
system_initializer = VSEPRInitializer()
initial_structure = system_initializer(definition)
compound_to_xyz.writer(monomer_CH3NH2_initial_structure_path, initial_structure)

Initializing compound C1H3N1H2
System initialization complete!


In [18]:
definition = xyz_to_compound_definition.reader(monomer_H2O_definition_path)
system_initializer = VSEPRInitializer()
initial_structure = system_initializer(definition)
compound_to_xyz.writer(monomer_H2O_initial_structure_path, initial_structure)

Initializing compound O1H2
System initialization complete!


## 2.2.x Geometry Minimization

That guess is looking a little rough, lets optimize it using electronic structure theory. While the PEF will be developed at the CCSD(T) level of theory, its not important that the geometry optimization be at the same level of theory as the reference energies will be calculated at, so lets use something cheaper like a high-quality DFT functional.

In [48]:
job = job_runner.queue_job("CH3NH2_minimization",
f"""
from mbfit.calculator import QchemCalculator
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system.descriptor import compound_to_xyz

calculator = QchemCalculator("wB97M-V", "AUG-cc-pVTZ", log_directory="{logs_dir}")
definition = xyz_to_compound_definition.reader("{monomer_CH3NH2_definition_path}")
initial_structure = xyz_to_compound_by_definition(definition).reader("{monomer_CH3NH2_initial_structure_path}")
optimized_structure, optimized_energy, optimization_log_path = calculator.optimize(initial_structure, num_threads=16, mem_mb=32000)
compound_to_xyz.writer("{monomer_CH3NH2_minimum_structure_path}", optimized_structure)
"""
)

print("Minimization job has been queued")

job.wait_until_started()

print("Minimization job has started execution")
job.wait_until_done()

print("Minimization job is done!")

Minimization job has been queued
Minimization job has started execution
Minimization job is done!


In [49]:
job = job_runner.queue_job("H2O_minimization",
f"""
from mbfit.calculator import QchemCalculator
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system.descriptor import compound_to_xyz

calculator = QchemCalculator("wB97M-V", "AUG-cc-pVTZ", log_directory="{logs_dir}")
definition = xyz_to_compound_definition.reader("{monomer_H2O_definition_path}")
initial_structure = xyz_to_compound_by_definition(definition).reader("{monomer_H2O_initial_structure_path}")
optimized_structure, optimized_energy, optimization_log_path = calculator.optimize(initial_structure, num_threads=16, mem_mb=32000)
compound_to_xyz.writer("{monomer_H2O_minimum_structure_path}", optimized_structure)
"""
)

print("Minimization job has been queued")

job.wait_until_started()

print("Minimization job has started execution")
job.wait_until_done()

print("Minimization job is done!")

Minimization job has been queued
Minimization job has started execution
Minimization job is done!


How calculate the CCSD(T) energies:

In [145]:
job = orca_runner.queue_job("CH3NH2_minimum_energy_calculation",
f"""
from mbfit.calculator import OrcaCalculator
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system.descriptor import compound_to_xyz

calculator = OrcaCalculator("DLPNO-CCSD(T)-F12", "cc-pVTZ-F12", log_directory="{logs_dir}", command_line_args = ["VeryTightSCF", "TightPNO", "PModel"], basis_args = {{"AuxC": "cc-pVTZ/C", "CABS": "cc-pVTZ-F12-CABS"}})
definition = xyz_to_compound_definition.reader("{monomer_CH3NH2_definition_path}")
optimized_structure = xyz_to_compound_by_definition(definition).reader("{monomer_CH3NH2_minimum_structure_path}")
optimized_energy, optimization_log_path = calculator.energy(initial_structure, num_threads=16, mem_mb=32000)
compound_to_xyz.writer("{monomer_CH3NH2_minimum_structure_path}", optimized_structure)
"""
)

print("Minimization job has been queued")

job.wait_until_started()

print("Minimization job has started execution")
job.wait_until_done()

print("Minimization job is done!")

Minimization job has been queued
Minimization job has started execution
Minimization job is done!


In [ ]:
job = orca_runner.queue_job("CH3NH2_minimum_energy_calculation",
f"""
from mbfit.calculator import OrcaCalculator
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system.descriptor import compound_to_xyz

calculator = OrcaCalculator("DLPNO-CCSD(T)-F12", "cc-pVTZ-F12", log_directory="{logs_dir}", command_line_args = ["VeryTightSCF", "TightPNO", "PModel"], basis_args = {{"AuxC": "cc-pVTZ/C", "CABS": "cc-pVTZ-F12-CABS"}})
definition = xyz_to_compound_definition.reader("{monomer_H2O_definition_path}")
optimized_structure = xyz_to_compound_by_definition(definition).reader("{monomer_H2O_minimum_structure_path}")
optimized_energy, optimization_log_path = calculator.energy(initial_structure, num_threads=16, mem_mb=32000)
compound_to_xyz.writer("{monomer_H2O_minimum_structure_path}", optimized_structure)
"""
)

print("Minimization job has been queued")

job.wait_until_started()

print("Minimization job has started execution")
job.wait_until_done()

print("Minimization job is done!")

## 2.2.2 Two-Body Scans

Now we make a few two-body scans. We will use these during future steps to validate the quality of the 2-body parts of the PEF.

Lets generate a few 2-body CH$_3$-NH$_2$ -- H$_2$O scans. We will make three scans.

In [108]:
dimer_CH3NH2_H2O_definition = xyz_to_compound_definition.reader(dimer_CH3NH2_H2O_definition_path)
dimer_CH3NH2_H2O = BasicCompound(dimer_CH3NH2_H2O_definition)

dimer_CH3NH2_H2O.fragments[0].set_xyz(monomer_CH3NH2_initial_structure_path)
dimer_CH3NH2_H2O.fragments[1].set_xyz(monomer_H2O_initial_structure_path)

monomer_CH3NH2 = dimer_CH3NH2_H2O.fragments[0]
monomer_H2O = dimer_CH3NH2_H2O.fragments[1]

H-bond between H on -NH$_2$ and O in H$_2$O:

In [110]:
scan_2b_1 = make_scan(
        system=dimer_CH3NH2_H2O,
        region1=monomer_CH3NH2,
        region2=monomer_H2O,
        center1=monomer_CH3NH2.atoms[5].point,
        center2=monomer_H2O.atoms[0].point,
        align1=[monomer_CH3NH2.atoms[5].point-monomer_CH3NH2.atoms[4].point, monomer_CH3NH2.atoms[5].point-monomer_CH3NH2.atoms[0].point],
        align2=[monomer_H2O.atoms[1].point-monomer_H2O.atoms[0].point + monomer_H2O.atoms[2].point-monomer_H2O.atoms[0].point, monomer_H2O.atoms[1].point - monomer_H2O.atoms[0].point],
        distances=numpy.arange(1.0, 10.5, 0.5)
)

compound_to_xyz.descriptor_sequence().writer(scan_2b_1_path, scan_2b_1)

H-bond between H on H$_2$O and N in -NH$_2$:

In [113]:
scan_2b_2 = make_scan(
        system=dimer_CH3NH2_H2O,
        region1=monomer_CH3NH2,
        region2=monomer_H2O,
        center1=monomer_CH3NH2.atoms[4].point,
        center2=monomer_H2O.atoms[1].point,
        align1=(monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[0].point + monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[5].point + monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[6].point, monomer_CH3NH2.atoms[0].point-monomer_CH3NH2.atoms[4].point),
        align2=(monomer_H2O.atoms[0].point-monomer_H2O.atoms[1].point, monomer_H2O.atoms[2].point-monomer_H2O.atoms[1].point),
        distances=numpy.arange(1.0, 10.5, 0.5)
)

compound_to_xyz.descriptor_sequence().writer(scan_2b_2_path, scan_2b_2)

H from -CH$_3$ approaching O in H$_2$O

In [115]:
scan_2b_3 = make_scan(
        system=dimer_CH3NH2_H2O,
        region1=monomer_CH3NH2,
        region2=monomer_H2O,
        center1=monomer_CH3NH2.atoms[1].point,
        center2=monomer_H2O.atoms[0].point,
        align1=(monomer_CH3NH2.atoms[1].point-monomer_CH3NH2.atoms[0].point, monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[1].point),
        align2=(monomer_H2O.atoms[1].point-monomer_H2O.atoms[0].point + monomer_H2O.atoms[2].point-monomer_H2O.atoms[0].point, monomer_H2O.atoms[1].point-monomer_H2O.atoms[0].point),
        distances=numpy.arange(1.0, 10.5, 0.5)
)

compound_to_xyz.descriptor_sequence().writer(scan_2b_3_path, scan_2b_3)

Now, lets perform the reference calculations for the scans.

In [149]:
job = orca_runner.queue_job("2B_scan_energies",
f"""
from mbfit.calculator import OrcaCalculator, QchemCalculator
from mbfit.system.parser import xyz_to_compound_definition, xyz_to_compound_by_definition
from mbfit.system.descriptor import compound_to_xyz, compound_to_commented_xyz

calculator = QchemCalculator("HF", "STO-3G", log_directory="{logs_dir}")
#calculator = OrcaCalculator("DLPNO-CCSD(T)-F12", "cc-pVTZ-F12", log_directory="{logs_dir}", command_line_args = ["VeryTightSCF", "TightPNO", "PModel"], basis_args = {{"AuxC": "cc-pVTZ/C", "CABS": "cc-pVTZ-F12-CABS"}})
definition = xyz_to_compound_definition.reader("{dimer_CH3NH2_H2O_definition_path}")
scan1_structures = xyz_to_compound_by_definition(definition).parser_sequence().reader("{scan_2b_1_path}")
2b_energies = []
for structre_index, structure in enumerate(scan1_structures):
    dimer_energy = optimized_structure, optimized_energy, optimization_log_path = calculator.optimize(structure, num_threads=16, mem_mb=32000)
    2b_energies.append(f"{{dimer_energy:+16.8f}}")
    print(f"Completed Energy Calculation {{structre_index}}")
compound_to_commented_xyz.descriptor_sequence().writer("{scan_2b_energies_1_path}", zip(*scan1_structures, *2b_energies))
"""
)

print("Minimization job has been queued")

job.wait_until_started()

print("Minimization job has started execution")
job.wait_until_done()

print("Minimization job is done!")

Minimization job has been queued
Minimization job has started execution
Minimization job is done!


## 2.2.3 Three-Body Scans

Now we make a few three-body scans. We will use these during future steps to validate the quality of the 2-body parts of the PEF.

Lets also consider a few 3-body structures. We will make three scans.

In [135]:
trimer_CH3NH2_H2O_definition = xyz_to_compound_definition.reader(trimer_CH3NH2_H2O_H2O_definition_path)
trimer_CH3NH2_H2O = BasicCompound(trimer_CH3NH2_H2O_definition)

trimer_CH3NH2_H2O.fragments[0].set_xyz(monomer_CH3NH2_initial_structure_path)
trimer_CH3NH2_H2O.fragments[1].set_xyz(monomer_H2O_initial_structure_path)
trimer_CH3NH2_H2O.fragments[2].set_xyz(monomer_H2O_initial_structure_path)

monomer_CH3NH2 = trimer_CH3NH2_H2O.fragments[0]
monomer_H2O_1 = trimer_CH3NH2_H2O.fragments[1]
monomer_H2O_2 = trimer_CH3NH2_H2O.fragments[2]

dimer = BasicMutableRegionManager(trimer_CH3NH2_H2O).make_region([monomer_CH3NH2, monomer_H2O_1])
transformer1 = SystemTransformer(monomer_CH3NH2)
transformer2 = SystemTransformer(monomer_H2O_1)

-NH$_2$ donating 2 H-bonds to 2 H$_2$Os, where one H$_2$O moves away:

In [136]:
transformer1.translate(translation=-monomer_CH3NH2.atoms[5].point)
transformer2.translate(translation=-monomer_H2O_1.atoms[0].point)
transformer1.align(vectors_to_align=[-monomer_CH3NH2.atoms[4].point, monomer_CH3NH2.atoms[0].point])
transformer2.align(vectors_to_align=[monomer_H2O_1.atoms[1].point + monomer_H2O_1.atoms[2].point, monomer_H2O_1.atoms[1].point])

# Make fixed H-bond 2.0 A long
transformer2.translate(x=2.0)

scan_3b_1 = make_scan(
        system=trimer_CH3NH2_H2O,
        region1=dimer,
        region2=monomer_H2O_2,
        center1=monomer_CH3NH2.atoms[6].point,
        center2=monomer_H2O_2.atoms[0].point,
        align1=[monomer_CH3NH2.atoms[6].point-monomer_CH3NH2.atoms[4].point, monomer_CH3NH2.atoms[6].point-monomer_CH3NH2.atoms[0].point],
        align2=[monomer_H2O_2.atoms[1].point-monomer_H2O_2.atoms[0].point + monomer_H2O_2.atoms[2].point-monomer_H2O_2.atoms[0].point, monomer_H2O_2.atoms[1].point - monomer_H2O_2.atoms[0].point],
        distances=numpy.arange(1.0, 10.5, 0.5)
)

compound_to_xyz.descriptor_sequence().writer(scan_3b_1_path, scan_3b_1)

-NH$_2$ donating 1 H-bond and accepting 1 H-bond, where the H$_2$O that is accepting an H-bond moves away.

In [139]:
transformer1.translate(translation=-monomer_CH3NH2.atoms[4].point)
transformer2.translate(translation=-monomer_H2O_1.atoms[1].point)
transformer1.align(vectors_to_align=[-monomer_CH3NH2.atoms[5].point-monomer_CH3NH2.atoms[6].point-monomer_CH3NH2.atoms[0].point, monomer_CH3NH2.atoms[0].point])
transformer2.align(vectors_to_align=[monomer_H2O_1.atoms[0].point, monomer_H2O_1.atoms[2].point])

# Make fixed H-bond 2.0 A long
transformer2.translate(x=2.0)

scan_3b_2 = make_scan(
        system=trimer_CH3NH2_H2O,
        region1=dimer,
        region2=monomer_H2O_2,
        center1=monomer_CH3NH2.atoms[6].point,
        center2=monomer_H2O_2.atoms[0].point,
        align1=[monomer_CH3NH2.atoms[6].point-monomer_CH3NH2.atoms[4].point, monomer_CH3NH2.atoms[6].point-monomer_CH3NH2.atoms[0].point],
        align2=[monomer_H2O_2.atoms[1].point-monomer_H2O_2.atoms[0].point + monomer_H2O_2.atoms[2].point-monomer_H2O_2.atoms[0].point, monomer_H2O_2.atoms[1].point - monomer_H2O_2.atoms[0].point],
        distances=numpy.arange(1.0, 10.5, 0.5)
)

compound_to_xyz.descriptor_sequence().writer(scan_3b_2_path, scan_3b_2)

-NH$_2$ donating 1 H-bond and accepting 1 H-bond, where the H$_2$O that is donating an H-bond moves away.

In [140]:
transformer1.translate(translation=-monomer_CH3NH2.atoms[5].point)
transformer2.translate(translation=-monomer_H2O_1.atoms[0].point)
transformer1.align(vectors_to_align=[-monomer_CH3NH2.atoms[4].point, monomer_CH3NH2.atoms[0].point])
transformer2.align(vectors_to_align=[monomer_H2O_1.atoms[1].point + monomer_H2O_1.atoms[2].point, monomer_H2O_1.atoms[1].point])

# Make fixed H-bond 2.0 A long
transformer2.translate(x=2.0)

scan_3b_3 = make_scan(
        system=trimer_CH3NH2_H2O,
        region1=dimer,
        region2=monomer_H2O_2,
        center1=monomer_CH3NH2.atoms[4].point,
        center2=monomer_H2O_2.atoms[1].point,
        align1=[monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[5].point + monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[6].point + monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[0].point, monomer_CH3NH2.atoms[4].point-monomer_CH3NH2.atoms[0].point],
        align2=[monomer_H2O_2.atoms[0].point-monomer_H2O_2.atoms[1].point, monomer_H2O_2.atoms[2].point - monomer_H2O_2.atoms[0].point],
        distances=numpy.arange(1.0, 10.5, 0.5)
)

compound_to_xyz.descriptor_sequence().writer(scan_3b_3_path, scan_3b_3)